In [3]:
!pip3 install --upgrade --quiet langchain langchain-openai langchain-community chromadb
!pip3 install --upgrade --quiet streamlit pandas

In [6]:
#Importação de documento pdf, vetorização, conexão com a OpenAI
import streamlit as st
from langchain.vectorstores import Chroma, FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

#Base de perguntas
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
import pandas as pd